In [ ]:
!unzip '/content/StructuredRegulatoryDocuments.zip'

Archive:  /content/StructuredRegulatoryDocuments.zip
   creating: StructuredRegulatoryDocuments/
  inflating: __MACOSX/._StructuredRegulatoryDocuments  
  inflating: StructuredRegulatoryDocuments/20.json  
  inflating: __MACOSX/StructuredRegulatoryDocuments/._20.json  
  inflating: StructuredRegulatoryDocuments/36.json  
  inflating: __MACOSX/StructuredRegulatoryDocuments/._36.json  
  inflating: StructuredRegulatoryDocuments/16.json  
  inflating: __MACOSX/StructuredRegulatoryDocuments/._16.json  
  inflating: StructuredRegulatoryDocuments/6.json  
  inflating: __MACOSX/StructuredRegulatoryDocuments/._6.json  
  inflating: StructuredRegulatoryDocuments/7.json  
  inflating: __MACOSX/StructuredRegulatoryDocuments/._7.json  
  inflating: StructuredRegulatoryDocuments/17.json  
  inflating: __MACOSX/StructuredRegulatoryDocuments/._17.json  
  inflating: StructuredRegulatoryDocuments/40.json  
  inflating: __MACOSX/StructuredRegulatoryDocuments/._40.json  
  inflating: StructuredRegulator

In [ ]:
!unzip '/content/ObliQA_train.zip'

Archive:  /content/ObliQA_train.zip
  inflating: ObliQA_train.json       


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_json('/content/StructuredRegulatoryDocuments/1.json')

In [ ]:
id=2
for i in range(id,41):
  filename='/content/StructuredRegulatoryDocuments/'+str(i)+'.json'
  df1=pd.read_json(filename)
  df=pd.concat([df,df1])

In [ ]:
df[df.isnull().any(axis=1)]

,ID,DocumentID,PassageID,Passage


In [ ]:
df

,ID,DocumentID,PassageID,Passage
0,bd35fb2d-4de6-48fb-ab3c-baead722854f,1,1.,INTRODUCTION
1,e563ad09-df80-435c-a497-eeec420efbc4,1,1.1,Jurisdiction
2,8edbc54b-4d98-4417-a4b7-3531b6f80f8c,1,1.1.1,
3,1389d0fa-5346-4953-8d52-45d7fd96c5e8,1,1.1.1.(1),The AML Rulebook is made in recognition of the...
4,23368989-55c6-4940-abc6-d7a8d2d1071b,1,1.1.1.(2),Nothing in the AML Rulebook affects the operat...
...,...,...,...,...
165,b3229a88-88fa-43e6-912f-30fb5763d6f6,40,SECTION IX.F.2.,The address of a Financial Institution with wh...
166,1f312be6-0e89-490f-b9c3-6ce0e8ae3234,40,SECTION IX.F.3.,An address that is provided subject to instruc...
167,029f87ab-4a4d-47af-9561-bc60ccff81b7,40,SECTION VIII.ANNEX 1,\nNON-REPORTING FINANCIAL INSTITUTIONS\nFor th...
168,1ca292b9-2154-49f5-bcf7-25e7a7898c4b,40,SECTION VIII.ANNEX 2,\nEXCLUDED ACCOUNTS\nFor the purposes of subpa...


In [ ]:
df['details']='DocumentID: '+df['DocumentID'].astype('str')+" | PassageID: "+df['PassageID'].astype('str')+" | Passage: "+df['Passage']

In [ ]:
def change_to_str(examples):
  examples=str(examples)
  return examples

df['DocumentID']=df['DocumentID'].map(change_to_str)
df['PassageID']=df['PassageID'].map(change_to_str)

In [ ]:
query_df=pd.read_json('/content/ObliQA_train.json')
dev_df=pd.read_json('/content/ObliQA_dev.json')

In [ ]:
from tqdm.auto import tqdm

progress_bar=tqdm(range(query_df.shape[0]))
train_df=pd.DataFrame()
train_passages=[]
for _,row in query_df.iterrows():
  question=row['Question']
  passage_id=[str(passageID) for passagedetails in (row['Passages']) for passagekey,passageID in passagedetails.items() if passagekey=='PassageID']
  document_id=[str(passageID) for passagedetails in (row['Passages']) for passagekey,passageID in passagedetails.items() if passagekey=='DocumentID']

  try:
    for pid,docid in zip(passage_id,document_id):
      matching_passage = df.loc[(df['PassageID'] == pid) & (df['DocumentID'] == docid), 'details'].values
      train_passages.append([question,matching_passage[0],1])
      false_passages = df.loc[~df['PassageID'].isin([passage_id]), 'details'].sample(n=2, random_state=42).values
      for passage in false_passages:
        train_passages.append([question,passage,0])
  except:
    pass

  progress_bar.update(1)

  0%|          | 0/22295 [00:00<?, ?it/s]

In [ ]:
#Trying for dataset for triplet loss
from tqdm.auto import tqdm

progress_bar=tqdm(range(query_df.shape[0]))
train_df=pd.DataFrame()
train_passages=[]
for _,row in query_df.iterrows():
  question=row['Question']
  passage_id=[str(passageID) for passagedetails in (row['Passages']) for passagekey,passageID in passagedetails.items() if passagekey=='PassageID']
  document_id=[str(passageID) for passagedetails in (row['Passages']) for passagekey,passageID in passagedetails.items() if passagekey=='DocumentID']

  try:
    for pid,docid in zip(passage_id,document_id):
      matching_passage = df.loc[(df['PassageID'] == pid) & (df['DocumentID'] == docid), 'details'].values
      false_passages = df.loc[~df['PassageID'].isin([passage_id]), 'details'].sample(n=1, random_state=42).values
      train_passages.append([question,matching_passage[0],false_passages[0]])
  except:
    pass

  progress_bar.update(1)

train_df = pd.DataFrame(train_passages, columns=["anchor", "positive", "negative"])

  0%|          | 0/22295 [00:00<?, ?it/s]

In [ ]:
#Trying for dataset for Multiple Negative symmetric Loss
from tqdm.auto import tqdm

progress_bar=tqdm(range(query_df.shape[0]))
train_df=pd.DataFrame()
train_passages=[]
for _,row in query_df.iterrows():
  question=row['Question']
  passage_id=[str(passageID) for passagedetails in (row['Passages']) for passagekey,passageID in passagedetails.items() if passagekey=='PassageID']
  document_id=[str(passageID) for passagedetails in (row['Passages']) for passagekey,passageID in passagedetails.items() if passagekey=='DocumentID']

  try:
    for pid,docid in zip(passage_id,document_id):
      matching_passage = df.loc[(df['PassageID'] == pid) & (df['DocumentID'] == docid), 'Passage'].values
      train_passages.append([question,matching_passage[0]])
  except:
    pass

  progress_bar.update(1)

train_df = pd.DataFrame(train_passages, columns=["anchor", "positive"])

  0%|          | 0/22295 [00:00<?, ?it/s]

In [ ]:
progress_bar=tqdm(range(dev_df.shape[0]))
val_df=pd.DataFrame()
dev_passages=[]
for _,row in dev_df.iterrows():
  question=row['Question']
  passage_id=[str(passageID) for passagedetails in (row['Passages']) for passagekey,passageID in passagedetails.items() if passagekey=='PassageID']
  document_id=[str(passageID) for passagedetails in (row['Passages']) for passagekey,passageID in passagedetails.items() if passagekey=='DocumentID']

  try:
    for pid,docid in zip(passage_id,document_id):
      matching_passage = df.loc[(df['PassageID'] == pid) & (df['DocumentID'] == docid), 'Passage'].values
      dev_passages.append([question,matching_passage[0]])
  except:
    pass

  progress_bar.update(1)

val_df = pd.DataFrame(dev_passages, columns=["anchor", "positive"])

  0%|          | 0/2788 [00:00<?, ?it/s]

In [ ]:
from sklearn.utils import shuffle

train_df = shuffle(train_df, random_state=42)
val_df = shuffle(val_df, random_state=42)

train_df.to_csv('train_obliqa.csv',index=False)
val_df.to_csv('dev_obliqa.csv',index=False)

In [ ]:
train_df = pd.DataFrame(train_passages, columns=["question", "details", "label"])

In [ ]:
train_df

,anchor,positive
0,"Under Rules 7.3.2 and 7.3.3, what are the two ...",DocumentID: 11 | PassageID: 7.3.4 | Passage: E...
1,**Best Execution and Transaction Handling**: W...,DocumentID: 3 | PassageID: 17.1.4 | Passage: T...
2,"How does the FSRA define and evaluate ""princip...",DocumentID: 11 | PassageID: 10.1.7.(2) | Passa...
3,How soon is the Registrar of Companies expecte...,DocumentID: 1 | PassageID: 15.9.1.(3) | Passag...
4,Can the ADGM provide examples of legal risks a...,DocumentID: 13 | PassageID: 4.14.3 | Passage: ...
...,...,...
29540,What are the reporting requirements for Financ...,DocumentID: 15 | PassageID: Part 3.6.(2) | Pas...
29541,What are the reporting requirements for Financ...,DocumentID: 15 | PassageID: Part 3.7.(1) | Pas...
29542,What are the penalties for non-compliance with...,DocumentID: 1 | PassageID: 2.Guidance.11. | Pa...
29543,If a communication references an investment th...,DocumentID: 17 | PassageID: Schedule 2.19.(3) ...


In [ ]:
from sklearn.utils import shuffle

train_df = shuffle(train_df, random_state=42)

In [ ]:
train_df.to_csv('MNSR_obliqa.csv',index=False)